In [4]:
import pandas as pd
import glob
from scipy import signal
import torch
import matplotlib.pyplot as plt
import numpy as np
from torch import nn,optim
from torch.nn import functional as F
from scipy import stats
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
import seaborn as sn
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.datasets import make_classification
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from tqdm import tqdm
from sklearn.utils import shuffle
from keras.utils import np_utils
from scipy.stats import norm,kurtosis,skew
import antropy as ant
from sklearn.datasets import make_classification
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [83]:
###
# All variables releated with data are located here to make an order
###
le=preprocessing.LabelEncoder()
LABELS=['running','standing','trotting','walking','grazing-eating']
PATH="C:\\Users\\emirc\\Desktop\\AAR\\dataHandled/*" #Refer the path having the csv files
OUTPUT_LABEL='ActivityEncoded'

REMOVE_COLUMNS=['Mx','My','Mz','Gx','Gy','Gz','G3D','M3D','A3D','segment'] #Columns to delete such as Gx Gy Gz
SUBJECTS=['Happy','Zafir','Drieku','Galoway','Patron','Bacardi']
TIME_PERIODS = 200
STEP_DISTANCE = 100
FILES = sorted(glob.glob(PATH))
N_FEATURES=3

In [1]:
def merge_activities(df):
    df['label']=df['label'].replace(to_replace=['trotting-natural'], value='trotting')
    df['label']=df['label'].replace(to_replace=['trotting-rider'], value='trotting')
    df['label']=df['label'].replace(to_replace=['galloping-rider'], value='running')
    df['label']=df['label'].replace(to_replace=['galloping-natural'], value='running')
    df['label']=df['label'].replace(to_replace=['walking-rider'], value='walking')
    df['label']=df['label'].replace(to_replace=['walking-natural'], value='walking')
    df['label']=df['label'].replace(to_replace=['grazing'], value='grazing-eating')
    df['label']=df['label'].replace(to_replace=['eating'], value='grazing-eating')
    result=df[df['label'].isin(LABELS)]
    return result

In [85]:
def createDataFrames():
    for subject in SUBJECTS:
        files=[]
        for file in FILES:
            if subject in file:
                files.append(file)
        df=pd.DataFrame()
        for file in files:
            csv=pd.read_csv(file,low_memory=False)
            df=df.append(csv)
            
        df.drop(REMOVE_COLUMNS, axis=1, inplace=True)
        df=df.dropna()
        df=merge_activities(df)
        df[OUTPUT_LABEL]=le.fit_transform(df['label'].values.ravel())
        df.to_csv('C:\\Users\\emirc\\Desktop\\AAR\\codes\\dataHandled\\'+subject+'.csv',index=False)

In [86]:
def createCleanCSV():
    df=pd.DataFrame()   
    for file in FILES:
        csv=pd.read_csv(file,low_memory=False)
        df=df.append(csv)
    
    df.drop(REMOVE_COLUMNS, axis=1, inplace=True)
    df=df.dropna()
    df=merge_activities(df)
    df[OUTPUT_LABEL]=le.fit_transform(df['label'].values.ravel())
    df.to_csv('C:\\Users\\emirc\\Desktop\\AAR\\codes\\dataHandled\\all.csv',index=False)
    return df

In [87]:
def getCSV(path, name):
    df=pd.DataFrame()
    csv=pd.read_csv(path+name+'.csv')
    df=df.append(csv)
    return df